In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np

path = '../data/WLDUSDT_1h.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2023-07-24 12:00:00,2.6478,2.6557,2.3910,2.5563,16849324.0
1,2023-07-24 13:00:00,2.5569,2.5923,2.4416,2.4534,10513084.0
2,2023-07-24 14:00:00,2.4534,2.4696,2.1716,2.2699,29348832.0
3,2023-07-24 15:00:00,2.2698,2.2927,2.1510,2.2274,21175385.0
4,2023-07-24 16:00:00,2.2262,2.3535,2.2132,2.2776,17822469.0


In [2]:
temp = df.ta.bbands()
df['BBANDS_U'] = temp['BBU_5_2.0']
df['BBANDS_L'] = temp['BBL_5_2.0']
df['BBANDS_M'] = temp['BBM_5_2.0']
temp = df.ta.cdl_pattern(name="closingmarubozu")
df['MARUBOZU'] = temp['CDL_CLOSINGMARUBOZU']
#singal_buy = MARUBOZU == 100 and close > DONCH_U 
#singal_sell = MARUBOZU == -100 and close < DONCH_L
df['Signal_Buy'] = np.where((df['close'] > df['BBANDS_L']) & (df['close'] < df['BBANDS_M']), 1, 0)
df['Signal_Sell'] = np.where((df['close'] < df['BBANDS_U']) & (df['close'] < df['BBANDS_M']), 1, 0)
df['Signal'] = df['Signal_Buy'].replace(1,2) + df['Signal_Sell'].replace(1,3)
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [3]:
df['Signal'].value_counts() , df['Signal_Buy'].value_counts(), df['Signal_Sell'].value_counts()

(Signal
 2    5100
 0       4
 Name: count, dtype: int64,
 Signal_Buy
 1    5100
 0       4
 Name: count, dtype: int64,
 Signal_Sell
 0    5104
 Name: count, dtype: int64)

In [4]:
df.ta.bbands()

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,14.462945,16.213334,17.963722,21.591957,0.530660
...,...,...,...,...,...
2511,182.689918,188.798004,194.906090,6.470499,0.492797
2512,179.729466,190.976004,202.222542,11.777960,0.921196
2513,178.872357,192.252002,205.631647,13.918862,0.787676
2514,180.643403,193.378000,206.112597,13.170678,0.514998


In [5]:
help(ta.donchian)

Help on function donchian in module pandas_ta.volatility.donchian:

donchian(high, low, lower_length=None, upper_length=None, offset=None, **kwargs)
    Donchian Channels (DC)
    
    Donchian Channels are used to measure volatility, similar to
    Bollinger Bands and Keltner Channels.
    
    Sources:
        https://www.tradingview.com/wiki/Donchian_Channels_(DC)
    
    Calculation:
        Default Inputs:
            lower_length=upper_length=20
        LOWER = low.rolling(lower_length).min()
        UPPER = high.rolling(upper_length).max()
        MID = 0.5 * (LOWER + UPPER)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        lower_length (int): The short period. Default: 20
        upper_length (int): The short period. Default: 20
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type

In [6]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,Dividends,Stock Splits,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
27,2014-04-02 04:00:00+00:00,14.666667,15.392667,14.536667,15.352667,161734500,0.0,0.0,15.448518,13.228816,14.338667,100.0,1,0,2,0.778497,28.262572
28,2014-04-03 04:00:00+00:00,15.353333,15.715333,14.800000,15.026667,163855500,0.0,0.0,15.658568,13.500899,14.579733,0.0,0,0,0,0.788271,26.048432
29,2014-04-04 04:00:00+00:00,15.067333,15.218000,14.083333,14.148667,170184000,0.0,0.0,15.659642,13.496092,14.577867,-100.0,0,0,0,0.813014,25.541932
30,2014-04-07 04:00:00+00:00,13.720667,14.413333,13.567333,13.834667,147832500,0.0,0.0,15.679142,13.451792,14.565467,0.0,0,0,0,0.815370,25.909197
31,2014-04-08 04:00:00+00:00,14.003333,14.432667,13.761333,14.364000,103339500,0.0,0.0,15.669163,13.421504,14.545334,0.0,0,0,0,0.805082,26.179022


In [7]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_24792\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,Dividends,Stock Splits,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,,,
2014-04-02 04:00:00+00:00,14.666667,15.392667,14.536667,15.352667,161734500,0.0,0.0,15.448518,13.228816,14.338667,100.0,1,0,2,0.778497,28.262572
2014-04-03 04:00:00+00:00,15.353333,15.715333,14.800000,15.026667,163855500,0.0,0.0,15.658568,13.500899,14.579733,0.0,0,0,0,0.788271,26.048432
2014-04-04 04:00:00+00:00,15.067333,15.218000,14.083333,14.148667,170184000,0.0,0.0,15.659642,13.496092,14.577867,-100.0,0,0,0,0.813014,25.541932
2014-04-07 04:00:00+00:00,13.720667,14.413333,13.567333,13.834667,147832500,0.0,0.0,15.679142,13.451792,14.565467,0.0,0,0,0,0.815370,25.909197
2014-04-08 04:00:00+00:00,14.003333,14.432667,13.761333,14.364000,103339500,0.0,0.0,15.669163,13.421504,14.545334,0.0,0,0,0,0.805082,26.179022


In [8]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [9]:
df['Signal'].value_counts()

Signal
0    2244
2     245
Name: count, dtype: int64

In [10]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 350
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 2:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            
cash = 10000
bt = Backtest(df, Scalping_Strategy, cash=cash, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2014-04-02 04:00...
End                       2024-02-21 05:00...
Duration                   3612 days 01:00:00
Exposure Time [%]                   48.814785
Equity Final [$]                 13279.037328
Equity Peak [$]                   14187.69001
Return [%]                          32.790373
Buy & Hold Return [%]             1168.639554
Return (Ann.) [%]                    2.912958
Volatility (Ann.) [%]               11.381647
Sharpe Ratio                         0.255935
Sortino Ratio                        0.389116
Calmar Ratio                         0.181588
Max. Drawdown [%]                  -16.041555
Avg. Drawdown [%]                   -3.606103
Max. Drawdown Duration     1017 days 00:00:00
Avg. Drawdown Duration      111 days 00:00:00
# Trades                                  244
Win Rate [%]                        45.491803
Best Trade [%]                      17.735194
Worst Trade [%]                     -24.79778
Avg. Trade [%]                    

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [12]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.2, 1.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.2, 1.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='SQN')
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: total: 12.2 s
Wall time: 12.3 s


Start                     2014-04-02 04:00...
End                       2024-02-21 05:00...
Duration                   3612 days 01:00:00
Exposure Time [%]                   13.057453
Equity Final [$]                 10391.068605
Equity Peak [$]                  10501.928127
Return [%]                           3.910686
Buy & Hold Return [%]             1168.639554
Return (Ann.) [%]                    0.389149
Volatility (Ann.) [%]                0.729659
Sharpe Ratio                          0.53333
Sortino Ratio                        0.712328
Calmar Ratio                         0.201477
Max. Drawdown [%]                   -1.931481
Avg. Drawdown [%]                   -0.234028
Max. Drawdown Duration     1322 days 00:00:00
Avg. Drawdown Duration       92 days 00:00:00
# Trades                                  244
Win Rate [%]                        56.557377
Best Trade [%]                       7.482393
Worst Trade [%]                    -11.235829
Avg. Trade [%]                    

In [13]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [14]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=1.0,TPSL_rate=0.2,size_multiplier_rate=0.1)>

In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.update_layout(title='Profit | '+symbol+' | '+ str(cash), yaxis_title='USDT', xaxis_title='Date')
fig.show()